<a href="https://colab.research.google.com/github/msuraj248/DeepLearning/blob/master/CNNmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow==1.15

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [31]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)

In [29]:
mnist = input_data.read_data_sets("mnist_data/",one_hot=True)

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [30]:
def init_weight(shape):
  init_random_dist = tf.truncated_normal(shape, stddev=1.0)
  return tf.Variable(init_random_dist)

In [33]:
def init_bias(shape):
  init_bias_val = tf.constant(value=0.1, shape=shape)
  return tf.Variable(init_bias_val)

In [44]:
def conv2d(x,w):
  return tf.nn.conv2d(x,filters = w,strides=[1,1,1,1],padding='SAME')

In [35]:
def max_pool_2by2(x):
  return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [37]:
def convolutional_layers(input_x, shape):
  w = init_weight(shape)
  b = init_bias([shape[3]])
  return tf.nn.relu(conv2d(input_x, w) + b)

In [38]:
def normal_full_layer(input_layer, size):
  input_size = int(input_layer.get_shape()[1])
  w = init_weight([input_size, size])
  b = init_bias([size])
  return tf.matmul(input_layer,w) + b

In [41]:
x = tf.placeholder(tf.float32, shape =[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None,10])

In [45]:
x_image = tf.reshape(x,shape=[-1,28,28,1])
convo_1 = convolutional_layers(x_image, shape=[6,6,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [50]:
convo_2 = convolutional_layers(convo_1_pooling,shape=[6,6,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [51]:
convo_2_flat = tf.reshape(convo_2_pooling, shape=[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [56]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,rate = hold_prob)

In [58]:
y_pred = normal_full_layer(full_one_dropout,10)
cross_entrophy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [59]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entrophy)

In [60]:
init = tf.global_variables_initializer()

In [61]:
steps = 100
with tf.Session() as sess:
  sess.run(init)
  for i in range(steps):
    batch_x , batch_y = mnist.train.next_batch(50) 
    sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
    # PRINT OUT A MESSAGE EVERY 100 STEPS
    if i%10 == 0:
      print('Currently on step {}'.format(i))
      print('Accuracy is:')
      # Test the Train Model
      matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
      acc = tf.reduce_mean(tf.cast(matches,tf.float32))

      print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
      print('\n')

Currently on step 0
Accuracy is:
0.098


Currently on step 10
Accuracy is:
0.098


Currently on step 20
Accuracy is:
0.098


Currently on step 30
Accuracy is:
0.098


Currently on step 40
Accuracy is:
0.098


Currently on step 50
Accuracy is:
0.098


Currently on step 60
Accuracy is:
0.098


Currently on step 70
Accuracy is:
0.098


Currently on step 80
Accuracy is:
0.098


Currently on step 90
Accuracy is:
0.098


